In [61]:
import pandas as pd
import numpy as np
from pandas import Timedelta
from datetime import timedelta
from sklearn.datasets import load_wine
from sklearn.ensemble import GradientBoostingClassifier

In [62]:
df_20_col = pd.read_parquet('../../models/KMeans/output/seatunnal_20col.parquet')
df_original = pd.read_parquet('../../Sonar/seatunnel_all_information.parquet')

In [63]:
df = pd.concat([df_20_col, df_original['total_time']], axis=1)

In [64]:
date = pd.to_timedelta(df['total_time'])
time_hours = date.dt.total_seconds() / 3600

df['hours'] = time_hours

In [65]:
df.columns

Index(['begin_D', 'begin_B', 'begin_CP', 'begin_C', 'begin_OOA', 'end_D',
       'end_B', 'end_CP', 'end_C', 'end_OOA', 'D_change', 'B_change',
       'CP_change', 'C_change', 'OOA_change', 'D_percent', 'B_percent',
       'CP_percent', 'C_percent', 'OOA_percent', 'total_time', 'hours'],
      dtype='object')

In [66]:
result_time  = []

for i in df['hours']:
    if i > 106 :
        result_time.append(0)
    elif (i>=25) & (i<=106):
        result_time.append(1)
    else:
        i <= 25
        result_time.append(2)
df['time_class'] = result_time

In [67]:
X = df.drop(['total_time', 'hours', 'time_class'], axis=1)
y = df['time_class']

In [71]:
X_train, X_test = X[:int(0.7 * len(X))], X[int(0.7 * len(X)):]
y_train, y_test = y[:int(0.7 * len(X))], y[int(0.7 * len(X)):]

In [72]:
int(0.7 * len(X))

747

In [73]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto' , random_state = 2))

In [74]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.19626168, 0.47663551, 0.18691589, 0.56338028, 0.56338028])

In [75]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([0, 0, 0, ..., 2, 2, 2])

In [76]:
confusion_matrix(y,y_pred)

array([[ 46,   0, 164],
       [ 63,   7, 185],
       [208,  24, 371]])

In [77]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred ,average='macro'))
print('recall_score = ', recall_score(y,y_pred ,average='macro'))
print('f1_score',f1_score(y,y_pred ,average='macro'))

precision =  0.2953982131617727
recall_score =  0.28725188251088163
f1_score 0.26145688827535757


In [78]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         1461
y_pred    1471
dtype: int64

In [79]:
from sklearn.model_selection import GridSearchCV

parameters = {

    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]

}
svc = SVC()
clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
clf.fit(X,y)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1], 'gamma': ['scale', 'auto'],
                         'random_state': [2]},
             scoring='f1_macro')

In [80]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2,

In [81]:
clf.best_estimator_

SVC(C=1, gamma='auto', random_state=2)

In [82]:
y_pred = clf.predict(X)
y_pred

array([1, 1, 1, ..., 2, 2, 2])

In [83]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2,

In [84]:
y_pred_clf = clf.best_estimator_.predict(X)
y_pred_clf

array([1, 1, 1, ..., 2, 2, 2])

In [90]:
confusion_matrix(y,y_pred_clf)

array([[125,  20,  65],
       [  4, 163,  88],
       [  3,  11, 589]])

In [85]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y,y_pred_clf,average='macro' ))
print('f1_score',f1_score(y,y_pred_clf,average='macro' ))

precision =  0.8603254738734217
recall_score =  0.7370788448049203
f1_score 0.777629496577449


In [86]:
clf.predict(X_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2,

In [87]:
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np

def f_importances(coef, names):

    names = np.arange(1)
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = X.columns
svm1 = svm.SVC(kernel='linear',C=1)
svm1.fit(X, y)

SVC(C=1, kernel='linear')

In [88]:
svm1.coef_

array([[ 0.00329506, -0.00234373, -0.00234373,  0.00984965,  0.00984965,
        -0.00501099, -0.00166709, -0.00166709, -0.01320706,  0.00575894,
         0.00830605, -0.00067663, -0.00067663,  0.02305671,  0.00409071,
        -0.01229337,  0.00129234,  0.00129234, -0.01276129,  0.00065972],
       [ 0.00922333, -0.00978999, -0.00978999,  0.00358026,  0.00358026,
         0.00113686,  0.00398511,  0.00398511, -0.00224608, -0.00064781,
         0.00808647, -0.0137751 , -0.0137751 ,  0.00582634,  0.00422807,
        -0.00166978,  0.00689407,  0.00689407, -0.00485839, -0.00096221],
       [ 0.00240492, -0.00323042, -0.00323042,  0.00096444,  0.00096444,
         0.00098496, -0.00104243, -0.00104243,  0.00021795,  0.00043718,
         0.00141996, -0.002188  , -0.002188  ,  0.00074649,  0.00052726,
         0.00058596,  0.00136394,  0.00136394, -0.00370232,  0.00026965]])